In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import genjax
genjax.pretty()

In [ ]:
## Load a video ##

import condorgmm
import condorgmm.data as data

video = (
    data.YCBVVideo.training_scene(2)
    # .downscale(CFG.downscale)
    .downscale(16)
)

import matplotlib.pyplot as plt
plt.imshow(video[1].rgb)

In [ ]:
from condorgmm.condor.interface.camera_tracking import initialize, update

frame=video[0]
    
camera_pose_world_frame=condorgmm.Pose(video[0].camera_pose)
gmm0, ccts0 = initialize(
    frame,
    CFG.n_gaussians,
    camera_pose_world_frame,
    tile_size_x=CFG.tile_size_x,
    tile_size_y=CFG.tile_size_y,
    max_n_gaussians_per_tile=CFG.max_n_gaussians_per_tile,
)
hypers = ccts0.hypers

In [ ]:
import jax
from condorgmm.condor.types import CondorGMMState
from condorgmm.condor.tiling import GridTiling, GridTilingConfig
import jax.numpy as jnp

def check_datapoint_in_tiling(
    st: CondorGMMState,
    idx: int
):
    tiling: GridTiling = st.matter.tiling
    assoc = st.datapoints.value.gaussian_idx[idx]
    gaussian_tile = tiling.gaussian_to_tile[assoc]
    dp_tile_y, dp_tile_x = tiling.config.datapoint_index_to_tile_index(idx)
    dp_tile = jnp.array([dp_tile_y, dp_tile_x])
    return jnp.all(jnp.abs(gaussian_tile - dp_tile) < 2)

st = ccts0.state
valid = jax.vmap(check_datapoint_in_tiling, in_axes=(None, 0))(
    st, jnp.arange(len(st.datapoints.value))
)
jnp.all(valid)

In [ ]:
import rerun as rr
from condorgmm.condor.rerun import log_state

# condorgmm.rr_init("step_inference_04")

# rr.set_time_sequence("inference", -1)
# log_state(ccts0.state, hypers)
# rr.log("depth_img/observation", rr.DepthImage(video[0].depth))
# rr.log("depth_img/inferred", rr.DepthImage(ccts0.state.datapoints.value.xyz[..., 2].reshape(frame.depth.shape)))

In [ ]:
# frame1 = video[100]
# import jax.numpy as jnp

# gmm1, ccts1, meta = update(
#     frame=frame1,
#     camera_pose_world_frame=condorgmm.Pose(frame1.camera_pose),
#     prev_state=ccts0.replace(hypers=ccts0.hypers.replace(crp_alpha_for_new_gaussian_in_step_model=jnp.array(0.1, dtype=float))),
#     n_sweeps_first_pass=1,
#     n_sweeps_second_pass=4,
#     run_second_pass=True,
#     log=True
# )
# # 1, 2 is probably enough.  possibly even 1, 1

In [ ]:
# idxs = ccts1.state.matter.tiling.tile_to_gaussians
# jnp.sum(
#     # jnp.where(ccts1.state.gaussian_has_assoc_mask[idxs], idxs, -1) != -1,
#     idxs > 0,
#     axis=-1
# )

In [ ]:
# idxs = ccts1.state.matter.tiling.tile_to_gaussians
# jnp.sum(
#     jnp.where(ccts1.state.gaussian_has_assoc_mask[idxs], idxs, -1) != -1,
#     axis=-1
# )

In [ ]:
# condorgmm.rr_init("step_inference_debug_00")

# rr.set_time_sequence("inference_step", -1)
# log_state(ccts0.state, hypers)
# rr.log("depth_img/observation", rr.DepthImage(video[0].depth))
# rr.log("depth_img/inferred", rr.DepthImage(ccts0.state.datapoints.value.xyz[..., 2].reshape(frame.depth.shape)))

In [ ]:
# from condorgmm.condor.rerun import log_state
# import rerun as rr
# frame = frame1
# for (i, label) in enumerate(meta.visited_states.all_labels):
#     rr.set_time_sequence("inference_step", i)
#     log_state(meta.visited_states.states[i], hypers)
#     rr.log("inference_move", rr.TextDocument(label))
#     rr.log("depth_img/observation", rr.DepthImage(frame.depth))
#     rr.log("depth_img/inferred", rr.DepthImage(meta.visited_states.states[i].datapoints.value.xyz[..., 2].reshape(frame.depth.shape)))

In [ ]:
condorgmm.rr_init("step_run_01")

In [ ]:
from tqdm import tqdm
import jax.numpy as jnp

def _f(x):
    return jnp.array(x, dtype=jnp.float32)
# ccts = ccts0.replace(
#     hypers=hypers.replace(
#         alpha_multiplier_for_evolved_gaussian=_f(5.0),
#         crp_alpha_for_new_gaussian_in_step_model=_f(5.0),
#     )
# )
ccts = ccts0

for i in tqdm(range(1000)):
    if i % 10 == 0:
        _, ccts = update(
            frame=video[i],
            camera_pose_world_frame=condorgmm.Pose(video[i].camera_pose),
            prev_state=ccts,
            n_sweeps_first_pass=CFG.n_steps_phase1,
            n_sweeps_second_pass=CFG.n_steps_phase2,
            run_second_pass=True
        )
    if i % 140 == 0:
        rr.set_time_sequence("inference", i+1)
        log_state(ccts.state, hypers)
        rr.log("depth_img/observation", rr.DepthImage(video[i].depth))
        rr.log("depth_img/inferred", rr.DepthImage(ccts.state.datapoints.value.xyz[..., 2].reshape(frame.depth.shape)))